The plotting and visualization of field data and Lagrangian particle trajectories is a post-processing step. Thus, we expect to have files for both fields and particle data on disk, which we need to load and parse in order to proceed.

For this small example, our field data are Eulerian hydrodynamics from the CMEMS ENWS dataset (year 2022, January 5st), and Parcels particles advected over the entire year 2022 with a common 2D-RK4 advection kernel without diffusion. The field was resample to a uniform, rectangular Arakawa A-grid for easier data access, and then stored as HDF5 dataset.

The first step here is to include the necessary packages and loading the field data.

In [391]:
# %matplotlib inline
%matplotlib notebook
import os
import h5py
import math
import xarray as xr
import numpy as np
from datetime import timedelta
from glob import glob

timestamp = 3600 * 24 * 26.25 # show the flow after 26 days 6 hours
Pn = 512        # limit the plot to max. 512 simultaneous particles
filedir = "./data"  # this is the local directory path on your system to the demo files 
particle_fpath = "simulate_ENWS_noMPI_p_n32768_365d_fwd_add_age.nc"
uvel_fpath = sorted(glob(os.path.join(filedir, '*U*.h5')))
vvel_fpath = sorted(glob(os.path.join(filedir, '*V*.h5')))
grid_fpath = os.path.join(filedir, 'grid.h5')

fU = None
fV = None
fX = None
fY = None
fT = None
speed = None
fU_ext = None
fV_ext = None
f_velmag_ext = None
extents = None
multifile = False
if len(uvel_fpath) > 0 and os.path.exists(uvel_fpath[0]) and len(vvel_fpath) > 0 and os.path.exists(vvel_fpath[0]) and os.path.exists(grid_fpath):
    # ===================================== #
    # ==== this has to be extended to  ==== #
    # === also process multifile fields === #
    # ==== Take over approach from     ==== #
    # ==== vtk_convert.                ==== #
    # ===================================== #
    # if os.path.exists(grid_fpath):
    f_grid = h5py.File(grid_fpath, "r")
    fX = f_grid['longitude'][()]
    fY = f_grid['latitude'][()]
    fT = f_grid['times'][()]
    extents = (fX.min(), fX.max(), fY.min(), fY.max())
    f_grid.close()
    del f_grid

    if len(uvel_fpath) > 1 and len(vvel_fpath) > 1:
        multifile |= True
    if len(uvel_fpath) > 0 and not multifile:
        uvel_fpath = uvel_fpath[0]
    if len(vvel_fpath) > 0 and not multifile:
        vvel_fpath = vvel_fpath[0]
    if not multifile:
        f_u = h5py.File(uvel_fpath, "r")
        fU = f_u['uo'][()]
        max_u_value = np.maximum(np.abs(fU.min()), np.abs(fU.max()))
        fU_ext = (-max_u_value, +max_u_value)
        f_u.close()
        del f_u
        f_v = h5py.File(vvel_fpath, "r")
        fV = f_v['vo'][()]
        max_v_value = np.maximum(np.abs(fV.min()), np.abs(fV.max()))
        fV_ext = (-max_v_value, +max_v_value)
        f_v.close()
        del f_v
    else:
        fU_ext = (100.0, -100.0)
        fV_ext = (100.0, -100.0)
        fU_array = []
        fV_array = []
        for fpath in uvel_fpath:
            f_u_i = h5py.File(fpath, "r")
            fUi = f_u_i['uo'][()]
            fUi = fUi[0] if fUi.shape[0]==1 else fUi
            max_u_value = np.maximum(np.abs(fUi.min()), np.abs(fUi.max()))
            fU_ext = (np.minimum(-max_u_value, fU_ext[0]), np.maximum(+max_u_value, fU_ext[1]))
            fU_array.append(fUi)
            f_u_i.close()
            del f_u_i
        fU = np.array(fU_array)
        del fU_array
        for fpath in vvel_fpath:
            f_v_i = h5py.File(fpath, "r")
            fVi = f_v_i['vo'][()]
            fVi = fVi[0] if fVi.shape[0]==1 else fVi
            max_v_value = np.maximum(np.abs(fVi.min()), np.abs(fVi.max()))
            fV_ext = (np.minimum(-max_v_value, fV_ext[0]), np.maximum(+max_v_value, fV_ext[1]))
            fV_array.append(fVi)
            f_v_i.close()
            del f_v_i
        fV = np.array(fV_array)
        del fV_array
    tdim = 0
    ydim = 1
    xdim = 2

if fU is not None and fV is not None:
    speed = fU ** 2 + fV ** 2
    speed = np.where(speed > .0, np.sqrt(speed), 0)
    f_velmag_ext = (speed.min(), speed.max())

Now, we load the particle data. The particle data, in this case, are stored in a NetCDF format. If you want to map this notebook to your own zArray Parcels output data, I recommend using Daan Reijnders' conversion script at https://github.com/daanreijnders/zarr2netcdf . The particles have standard attributes of RK4-advected, non-diffusive Parcels particles.

In [392]:
data_xarray = xr.open_dataset(os.path.join(filedir, particle_fpath))
np.set_printoptions(linewidth=160)
sec_per_day = 86400.0
ns_per_sec = np.timedelta64(1, 's')  # nanoseconds in an sec
time_array = (data_xarray['time'].data-np.datetime64(0, 'ns')) / ns_per_sec

After loading the particle file, we sample _N_ (with _N=512_ here) particle, in order to not overload the plot.

In [393]:
N = data_xarray['lon'].data.shape[0]
Pn = min(Pn, N)
tN = data_xarray['lon'].data.shape[1]
indices = np.random.randint(0, N - 1, Pn, dtype=int) 
time_since_release_nc = (time_array.transpose() - time_array[:, 0])  # substract the initial time from each timeseri
sim_dt_nc = time_since_release_nc[1,0] - time_since_release_nc[0,0]
pX = data_xarray['lon'].data
pY = data_xarray['lat'].data
base_px = np.array([extents[0], extents[0], extents[1], extents[1]])
base_py = np.array([extents[2], extents[3], extents[2], extents[3]])

Next, we interpolate the field at the timestep we aim to investigate.

In [394]:
def time_index_value(tx, _ft):
    # expect ft to be forward-linear
    ft = _ft
    if isinstance(_ft, xr.DataArray):
        ft = ft.data
    f_dt = ft[1] - ft[0]
    if type(f_dt) is not np.float64:
        f_dt = timedelta(f_dt).total_seconds()
    f_interp = tx / f_dt
    ti = int(math.floor(f_interp))
    return ti


def time_partion_value(tx, _ft):
    # expect ft to be forward-linear
    ft = _ft
    if isinstance(_ft, xr.DataArray):
        ft = ft.data
    f_dt = ft[1] - ft[0]
    if type(f_dt) is not np.float64:
        f_dt = timedelta(f_dt).total_seconds()
    f_interp = tx / f_dt
    f_t = f_interp - math.floor(f_interp)
    return f_t

signed_plot = False # for now, we only deal with absolute velocity magnitude
tx = timestamp
tx = math.fmod(tx, fT[-1])
ti0 = time_index_value(tx, fT)
tt = time_partion_value(tx, fT)
ti1 = 0
if ti0 < (len(fT)-1):
    ti1 = ti0+1
else:
    ti1 = 0

fu_show = (1.0 - tt) * fU[ti0] + tt * fU[ti1]
fu_sign = np.sign(fu_show)

fv_show = (1.0 - tt) * fV[ti0] + tt * fV[ti1]
fv_sign = np.sign(fv_show)

fs_show = fu_show ** 2 + fv_show ** 2
fs_show = np.where(fs_show > np.finfo(fs_show.dtype).eps, np.sqrt(fs_show), .0)
if signed_plot:                 # we compute the combined sign of the flow direction; purpose becomes apparent later
    fsign = fu_sign + fv_sign
    fsign = np.sign(fsign)      # np.where(fsign < 0., -1.0, 1.0)
    fs_show= fs_show * fsign
    f_velmag_ext = (-f_velmag_ext[1], f_velmag_ext[1])

px_t_0 = np.array(pX[:, ti0])
px_t_0 = np.take_along_axis(px_t_0, indices, axis=0)
py_t_0 = np.array(pY[:, ti0])
py_t_0 = np.take_along_axis(py_t_0, indices, axis=0)

px_t_1 = np.array(pX[:, ti1])
px_t_1 = np.take_along_axis(px_t_1, indices, axis=0)
py_t_1 = np.array(pY[:, ti1])
py_t_1 = np.take_along_axis(py_t_1, indices, axis=0)

px_show = (1.0 - tt) * px_t_0 + tt * px_t_1
py_show = (1.0 - tt) * py_t_0 + tt * py_t_1
px_show = np.hstack((base_px, px_show))
py_show = np.hstack((base_py, py_show))
pd_show = np.ones(px_show.shape[0], dtype=px_show.dtype)

Now having prepared all the data for visualization, we can proceed with plotting the data using matplotlib, and Cartopy. We will overlay the velocity-magnitude field data with semi-transparent, elliptical blobs for the particles.

In [395]:
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from code import colour_scales
from code import TransparentEllipses

psize = 0.1
ls = psize * 0.05
alpha = 0.3
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 9))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap="gist_rainbow", interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)

# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>

We can improve this basic plot with a distinct colour for terrestrial areas, and even overlay bathymetric data to the plot. In order to do that, we will need to manage the individual layers of the plot using the _zorder_ function parameter.

In [396]:
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap="gist_rainbow", interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

/tmp/ipykernel_35834/2673610651.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  datafig = plt.figure(figsize=(12, 8))


<IPython.core.display.Javascript object>

## Interlude on rainbow colour maps, the IPCC visual style guide and cmocean

After understanding now the drawbacks of rainbow colour schemes, let's explore the _cmocean_ package and its features. _cmocean_ provides a range of thematic colour maps. For our purpose, as we are plotting _velocity magnitude_ (i.e. _speed_), we can employ the _speed_ colourmap here.

In [397]:
import cmocean
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap=cmocean.cm.speed, interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>

The resulting plot is already much cleaner. Some advantages in this case are:
* it is bitonal, only using two base-tone colours to intermix values
* it is continuous, without artificial jumps in the colour gradient
* it has a defined zero-point that matches the tone of the adjacent landmass

What this colour map still misses is a thematic association to '_the liquid ocean_', which is base tones don't match well to. Luckily, _cmocean_ also provide another colour map for velocity information named _tempo_, which is more suitable to this case.

In [398]:
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap=cmocean.cm.tempo, interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>

## Interlude on perceptual guidelines for colour maps

We have just discussed perceptual principles and their impact on the creation and the selection of adequate colour maps for individual case studies. When looking at the data aspect, it is important to characterize the attribute one wants to display.

In our current case, we deal with velocity magnitude, which is a linear-continuous, monotonously-increasing, lower-bound attribute. According to simplicity criteria, and in-line with the IPCC visual style guide, this attribute would benefit from a linear, monochromatic colour map with a base tone suitable for ocean liquids (i.e. blue). The attribute then modulates the _saturation_ of this base tone (from 0 saturation to full saturation) through the colour map.

In [399]:
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap="Blues", interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>

Now, we can improve on this visualization by adding directional information back to the plotted velocities. In the two hydrodynamic flow scenario, we have the U- and V-components, which are both scalar, signed, real-value quantities for the strength of the current in meridional- and zonal direction. Therefore, we can define a net-positive and net-negative velocity magnitude based on the sign-sum of both velocity components. This is illustrated as such:

![Alt text](signed_speed.png "Speed Quadrants")

The resulting quantity is now a linear, divergent attribute with a distinct zero-point. These attributes are best plotted via _divergent colour maps_. The most prominent one is _red-to-blue_.

In [400]:
signed_plot = True # for now, we only deal with absolute velocity magnitude
fs_show = fu_show ** 2 + fv_show ** 2
fs_show = np.where(fs_show > np.finfo(fs_show.dtype).eps, np.sqrt(fs_show), .0)
if signed_plot:                 # we compute the combined sign of the flow direction; purpose becomes apparent later
    # fsign = fu_sign + fv_sign
    fsign = fu_show + fv_show
    fsign = np.sign(fsign)      # np.where(fsign < 0., -1.0, 1.0)
    fs_show= fs_show * fsign
    fmax = np.maximum(np.absolute(fs_show.min()), np.absolute(fs_show.max()))
    # f_velmag_ext = (-f_velmag_ext[1], f_velmag_ext[1])
    f_velmag_ext = (-fmax, fmax)
    # f_velmag_ext = (fs_show.min(), fs_show.max())
    print( (fs_show.min(), fs_show.max()) )
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap="RdBu", interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()


(-0.30233410663318705, 0.23510670960044028)


<IPython.core.display.Javascript object>

A criticism of the _red-to-blue_ map is that, from a perceptual perspective, it is best to pick two base tones that are _complementary colour tones_ on the colour wheel. Red and blue are not complementary colours. This means that the zero-point becomes a visually sharper separator than what the underlying data suggest. This can be fixed by using a divergent, complementary colour map, such as _pink-to-neongreen_ (i.e. 'PiYG').

In [401]:
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap="PiYG", interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>

Lastly, we want to accommodate for audiences and viewer who may have a colour-vision impediment, such as _deuteranopia_ (i.e. red-green 'colour blindness'). In PiYG, the pink tone is dominated by the red values, while neongreen is decisively 'green' - which, of course, directly leads to a problem for people with red-green deuteranopia. Therefore, in order to account for the colour deficiencies, we can decide for a robust divergent, complementary colour map, such as _brown-to-teal_ (i.e. 'BrBG').

In [402]:
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0
# ==== plot the field data, the particle data and the figure's colour scale ==== #
cs1_i = dataaxis.imshow(fs_show, extent=plot_extent, transform=crsdata, cmap="BrBG", interpolation='hermite', norm=colors.Normalize(vmin=f_velmag_ext[0], vmax=f_velmag_ext[1]), zorder=zorder_s+1, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap='Greys_r', norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=zorder_s+3, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_velmag_bounds = []
# ======= setup a linear legend ======= #
increment_sbar = (f_velmag_ext[1] - f_velmag_ext[0]) / 6.0
bval = f_velmag_ext[0] - increment_sbar
while bval < f_velmag_ext[1]:
    bval += increment_sbar
    cbar_h5a_velmag_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_velmag = datafig.add_axes([0.1, 0.055, 0.8, 0.02])
cbar_h5a_velmag = plt.colorbar(cs1_i, cax=ax_cbar_h5a_velmag, ticks=cbar_h5a_velmag_bounds, orientation='horizontal')
title = "absolute velocity magnitude [m/s]"
ax_cbar_h5a_velmag.set_xlabel(title)
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>

## Interlude: Multivariate, divergent colour maps

We now had a look on the theory of multivariate colour mapping. The approach relies heavily upon map layers. The colour composition itself associates each _mode_ (i.e. each dimension) of a vectorial attribute with a base tone along the quadrant of one of the primary base colours. This avoids colour deficiency conflicts. As we have three primary base colours (i.e. red, green and blue), there are three different versions of the perceptual colour map. For our case, we will choose the _blue_ base quadrant.

We will first setup the principal figure, which is identical to univariate colour maps we looked at before.

Next, we create the multivariate field visualisation. Herefore, we use the prepared colour mapping classes, developed for the paper by Kehl et al. "Perceptual multivariate visualisation of volumetric Lagrangian fluid-flow processes" (DOI: https://doi.org/10.3389/fenvs.2022.941910 ). Those are in the Python module _multivariate_colour_palettes.py_ of the _code_ package, which is delivered as part of this notebook. In contrast to previous plots, we also need to create another layer for the background colour. Furthermore, we adapt the colour map of the particles to a tone map that does not conflict nor coincide with the field basemap.

Lastly, we add the topographic features, adapt the figure's colour scale to the multivariate map, and then plot the figure. As we now have 2 different colour maps, we prefer to plot the colours scales left- and right of the figure instead of below it.

In [403]:
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- PLOT ---- ---- ---- ---- ---- #
# ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- #
datafig = plt.figure(figsize=(12, 8))
midlon = extents[0] + ((extents[1] - extents[0]) / 2.0)
midlat = extents[2] + ((extents[3] - extents[2]) / 2.0)
# # ==== setup Cartopy environment, in terms of projection system ==== #
crsaxis = ccrs.PlateCarree()
crsdata = crsaxis
dataaxis = datafig.add_axes(rect=tuple([0.14, 0.11, 0.75, 0.78]), projection=crsaxis)
dataaxis.set_extent([extents[0], extents[1], extents[2], extents[3]], crs=crsaxis)
plot_extent = (extents[0], extents[1], extents[2], extents[3])
zorder_s = 0

# ==== ==== ==== ==== ==== ==== ==== ==== ==== #
# ==== setup the multivariate colour map  ==== #
# ==== ==== ==== ==== ==== ==== ==== ==== ==== #
cmap = colour_scales['blue_quadrant_palette_greybase_alpha']
cs0_i = dataaxis.imshow(np.zeros(fu_show.shape), extent=plot_extent, transform=crsdata, cmap="binary", interpolation='bilinear', zorder=zorder_s+0, origin='lower')
cs1_i = dataaxis.imshow(fu_show, extent=plot_extent, transform=crsdata, cmap=cmap['U']['colour_scale'], interpolation='hermite', norm=colors.Normalize(vmin=fU_ext[0], vmax=fU_ext[1]), zorder=zorder_s+1, origin='lower')
cs2_i = dataaxis.imshow(fv_show, extent=plot_extent, transform=crsdata, cmap=cmap['V']['colour_scale'], interpolation='hermite', norm=colors.Normalize(vmin=fV_ext[0], vmax=fV_ext[1]), zorder=zorder_s+2, origin='lower')
cs_h5a_pts = TransparentEllipses(px_show, py_show, s=psize, linewidths=ls, values=pd_show, cmap=colour_scales['white_to_red_pallete_opaque']['Any']['colour_scale'], norm=colors.LogNorm(vmin=0.001, vmax=1.0), zorder=4, transform=dataaxis.transData, edgecolors='black')
cs_h5a_pts_coll = cs_h5a_pts.get_collection()
dataaxis.add_collection(cs_h5a_pts_coll)

# ==== ==== ==== ==== ==== ==== ==== #
# ==== setup the colour scales  ==== #
# ==== ==== ==== ==== ==== ==== ==== #
# =============================================== #
# ==== Plotting geographic added information ==== #
# ==== on top of image or pcolormesh -       ==== #
# ==== necessarily.                          ==== #
# =============================================== #
dataaxis.add_feature(cfeature.LAND, color="khaki", zorder=zorder_s+3, edgecolor='black')
dataaxis.coastlines(color='black', linewidth=1, zorder=zorder_s+4)
dataaxis.gridlines(draw_labels=False, color='gray', linestyle='--')

cbar_h5a_u_bounds = []
# ======= setup a linear legend ======= #
increment_ubar = (fU_ext[1] - fU_ext[0]) / 6.0
bval = fU_ext[0] - increment_ubar
while bval < fU_ext[1]:
    bval += increment_ubar
    cbar_h5a_u_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_u = datafig.add_axes([0.055, 0.1, 0.02, 0.8])
ax_cbar_h5a_u.set_facecolor("white")
cbar_h5a_u = plt.colorbar(cs1_i, cax=ax_cbar_h5a_u, ticks=cbar_h5a_u_bounds, orientation='vertical', label="u-velocity magnitude [m/s]")
ax_cbar_h5a_u.yaxis.set_ticks_position('left')
ax_cbar_h5a_u.yaxis.set_label_position('right')

cbar_h5a_v_bounds = []
# ======= setup a linear legend ======= #
increment_vbar = (fV_ext[1] - fV_ext[0]) / 6.0
bval = fV_ext[0] - increment_vbar
while bval < fV_ext[1]:
    bval += increment_vbar
    cbar_h5a_v_bounds.append(float(int(bval*1000))/1000.)
# ============================= #
ax_cbar_h5a_v = datafig.add_axes([0.915, 0.1, 0.02, 0.8])
ax_cbar_h5a_v.set_facecolor("white")
cbar_h5a_v = plt.colorbar(cs2_i, cax=ax_cbar_h5a_v, ticks=cbar_h5a_v_bounds, orientation='vertical', label="v-velocity magnitude [m/s]")
ax_cbar_h5a_v.yaxis.set_label_position('left')
                
# ==== set the plot title and plot the figure ==== #
dataaxis.set_title("Simulation - NetCDF data - t = %5.1f d" % (timestamp / sec_per_day))
plt.show()

<IPython.core.display.Javascript object>